# Instructions
This notebook is designed to run scoring on a saved model in Snowflake.
Follow the steps in each cell to perform the inference.

In [ ]:
# Import necessary libraries
from snowflake.snowpark import Session
import pandas as pd


In [ ]:
-- Create Stream for Real-Time Inference
CREATE OR REPLACE STREAM ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2 ON TABLE ANALYTICS_TOOL_KIT.PUBLIC.SCORING_TABLE SHOW_INITIAL_ROWS = TRUE APPEND_ONLY = TRUE;


In [ ]:
-- Select data from the Stream
SELECT * FROM ANALYTICS_TOOL_KIT.PUBLIC.MYSICKSTREAM2 WHERE METADATA$ACTION = 'INSERT';


In [ ]:
# Execute the query and load the results into a Snowpark DataFrame
# Ensure the session is already initialized in the notebook environment
df_snowflake = session.sql('SELECT * FROM {database}.{schema}.{stream_name} WHERE METADATA$ACTION = 'INSERT'').to_pandas()
df_snowflake.head()
